In [ ]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np  # Import numpy for numerical operations
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Import StandardScaler for feature scaling and LabelEncoder for encoding target labels
from xgboost import XGBClassifier  # Import XGBClassifier for XGBoost model
from lightgbm import LGBMClassifier  # Import LGBMClassifier for LightGBM model
from catboost import CatBoostClassifier  # Import CatBoostClassifier for CatBoost model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score  # Import specific metrics for model evaluation
import warnings  # Import warnings to manage warnings
import category_encoders as ce  # Import category_encoders for encoding categorical features
import optuna
# Ignore warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [ ]:
path = r'C:\Users\User\Desktop\Rashad\DATA\dataset.csv'

# Load dataset from Excel file
df = pd.read_csv(path)

# Set option to display all columns
pd.set_option('display.max_columns', None)

#Display the loaded dataset
df

In [ ]:
df.select_dtypes('object').columns

In [ ]:
models = ["xgboost", "lightgbm", "catboost", "catboost_custom"]

# Create copies of the DataFrame for different preprocessing approaches
df_boost = df.copy()
df_cat = df.copy()

# Iterate through each model in the list
for model in models:            
    if model in ["xgboost", "lightgbm", "catboost"]:
        # Preprocessing for tree-based models
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_boost.columns.tolist()
        all_columns.remove('Target')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_boost[all_columns] = encoder.fit_transform(df_boost[all_columns], df_boost['Target'])
             
    elif model ==  "catboost_custom":
        # Preprocessing for CatBoost with custom handling of missing values
        
       
        columns_to_fill = cat_features=['Income_type', 'Education_type', 'Family_status', 'Housing_type',
       'Occupation_type']
        df_cat[columns_to_fill] = df_cat[columns_to_fill].fillna('Missing Value')
        
        
# Split data into features (X) and target (y) for each preprocessing approach
x_boost = df_boost.drop(columns=['Target'])
y_boost = df_boost['Target']
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(x_boost, y_boost, test_size=0.3, random_state=42)

x_cat = df_cat.drop(columns=['Target'])
y_cat = df_cat['Target']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(x_cat, y_cat, test_size=0.3, random_state=42)

In [ ]:
numerical_cols = df_cat.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('Target')
# Perform Weight of Evidence Encoding on numerical columns
cat_encoder = ce.WOEEncoder(cols=numerical_cols)
df_cat[numerical_cols] = cat_encoder.fit_transform(df_cat[numerical_cols], df_cat['Target'])

In [ ]:
models = []

xgb_model_def = XGBClassifier()
lgb_model_def = LGBMClassifier()
catboost_model_def = CatBoostClassifier()
catboost_model_custom = CatBoostClassifier(cat_features=['Income_type', 'Education_type', 'Family_status', 'Housing_type',
       'Occupation_type'])

models.extend([
    ('XGBoost', xgb_model_def),
    ('LightGBM', lgb_model_def),
    ('CatBoost', catboost_model_def),
    ('CatBoost_Custom', catboost_model_custom),
])

In [ ]:
def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the given model on the training and testing data.

    Parameters:
    model_name (str): Name of the model for display purposes.
    model : Machine learning model object.
    X_train : Features of the training data.
    y_train : Target labels of the training data.
    X_test : Features of the testing data.
    y_test : Target labels of the testing data.

    Returns:
    float, float: Gini coefficients calculated from the model's predictions on training and testing data.
    """

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict labels and probabilities on the testing data
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1]

    # Predict labels and probabilities on the training data
    y_train_pred = model.predict(X_train)
    y_train_prob = model.predict_proba(X_train)[:, 1]

    # Calculate ROC AUC and Gini coefficient for testing data
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1

    # Calculate ROC AUC and Gini coefficient for training data
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1

    # Calculate confusion matrix and classification report for testing data
    confusion_matrix_test_result = confusion_matrix(y_test, y_test_pred)
    classification_report_test_result = classification_report(y_test, y_test_pred)

    # Calculate confusion matrix and classification report for training data
    confusion_matrix_train_result = confusion_matrix(y_train, y_train_pred)
    classification_report_train_result = classification_report(y_train, y_train_pred)

    # Print model performance metrics
    print(f'Model Performance for {model_name}')
    print('Gini prob for testing data is', gini_test_prob * 100)
    print('Gini prob for training data is', gini_train_prob * 100)
    print('Classification Report for Testing Data:')
    print(classification_report_test_result)
    print('Confusion Matrix for Testing Data:')
    print(confusion_matrix_test_result)
    print('Classification Report for Training Data:')
    print(classification_report_train_result)
    print('Confusion Matrix for Training Data:')
    print(confusion_matrix_train_result)

    return gini_train_prob, gini_test_prob

In [ ]:
# Initialize the DataFrame to store Gini coefficients
gini_df = pd.DataFrame(columns=['Model', 'Gini_train_prob', 'Gini_test_prob'])

# Iterate through each model in the list of models
for model_name, model in models:
    # Train and evaluate the model, and calculate the Gini coefficient
    if model_name == 'CatBoost_Custom':
        gini_train_prob, gini_test_prob = train_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
    else:
        gini_train_prob, gini_test_prob = train_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
    
    # Add model name and Gini coefficients to the DataFrame
    gini_df = pd.concat([gini_df, pd.DataFrame({'Model': [model_name], 'Gini_train_prob': [gini_train_prob], 'Gini_test_prob': [gini_test_prob]})], ignore_index=True)

# Sort the DataFrame by Gini coefficient for testing data in descending order
gini_df_sorted = gini_df.sort_values(by='Gini_test_prob', ascending=False)

In [ ]:
gini_df_sorted